In [ ]:
import os
import cv2
import pandas as pd
from shapely.geometry import Polygon, mapping
from osgeo import gdal
import rasterio
import geopandas as gpd
import numpy as np
import tqdm
import rasterio.mask
import torch
import math
dim=700

In [ ]:
def correct_orientation(img_rgb,dim):
    img_rgb=cv2.imread(img_rgb)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    template = cv2.imread("pattern.png",0)
    w, h = template.shape[::-1]
    image_ro=img_gray.copy()
    angulo=0
    an=[]
    le=0
    angulo_f=0
    for i in range(0,90,10):
        angulo=i
        M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
        image_ro = cv2.warpAffine(img_gray, M, (dim,dim))
        res = cv2.matchTemplate(image_ro,template,cv2.TM_CCOEFF_NORMED)
        threshold =.5
        loc = np.where( res >= threshold)
        com=len(loc[0])
        if com>0:
            an.append(angulo)
            if le<com:
                le=com
                angulo_f=angulo
    return angulo_f
    

In [ ]:
def verificacion(im):
    hsv=cv2.cvtColor(im,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(hsv,(12,30,0),(160,232,160))
    verde=int((np.sum(mask)/256**2/255)*100)
    return verde 

In [ ]:
def vector2xy(vector,dim=700,nameimg="image",angle=0):
    s=[]
    for v in vector:
        str_v=(str(v).replace("tensor(","").replace("[","").replace("]","").replace(".)","").replace(")","").replace("(","").replace("']","").replace("'","").strip().split(","))
        h,w=dim,dim
        x1 = int( float(str_v[1]) * w )
        y1 = int( float(str_v[2]) * h )
        xw = int( float(str_v[3]) * w /2)
        yw = int( float(str_v[4]) * h /2)
        start_point_im = ((x1 - xw), (y1 - yw))
        end_point_im   = ((x1 + xw), (y1 + yw))
        start_point_100 = ((x1 - xw)/w, (y1 - yw)/h)
        end_point_100   = ((x1 + xw)/w, (y1 + yw)/h)
        area=xw*yw
        if str(str_v[0])=="0":
            tipo="casa"
        else:
            tipo="terreno"
        if int(xw)!=0 and int(yw)!=0 and (xw/yw<=4  and yw/xw<=4 ):
            s.append([tipo,start_point_im,end_point_im,start_point_100,end_point_100,area,nameimg])
    df_cache=pd.DataFrame(s,columns=["Tipo","start_point_im","end_point_im","start_point_100","end_point_100","area","imagen"])
#     print(np.min(df_cache["area"]),np.max(df_cache["area"]))
    try:
        df_cache["area"]=[int((x-np.min(df_cache["area"]))/(np.max(df_cache["area"])-np.min(df_cache["area"]))*100) for x in df_cache["area"]]
    except:
        pass
    df_cache.drop_duplicates().reset_index(drop=True,inplace=True)
#   
    return df_cache

In [ ]:
def imshow_detect(df_cache,imagen_n,nameimg="image"):
    for i in range(len(df_cache)):
            if df_cache["Tipo"][i]=="casa":
                cv2.rectangle(imagen_n,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,255-int(df_cache.loc[i,"area"]),0),2)
            else:
                cv2.rectangle(imagen_n,df_cache["start_point_im"][i],df_cache["end_point_im"][i],(0,0,255),2)
    imagen_n=cv2.resize(imagen_n,(1024,1024))
    cv2.imshow(nameimg,imagen_n)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [ ]:
def rotacion_detect(startpoint,endpoint,angle,proyeciones,dim=700):
    min_y,min_x,z=np.min((proyecciones,proyecciones),axis=1)[0]
    max_y,max_x,z=np.max((proyecciones,proyecciones),axis=1)[0]
    min_y,min_x,max_y,max_x,proyecciones
    tipos=["casa","terreno"]
    y1,x1=startpoint
    y2,x2=endpoint
    x1,y1=x1*2-1,y1*2-1
    x2,y2=x2*2-1,y2*2-1
    angle=angle*math.pi/180
    #x_p, y_p son los puntos de un rectangulo en el orden inverso al manecillas del reloj
    x1p=max_x-((x1*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y1p=min_y+((x1*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x2p=max_x-((x2*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y2p=min_y+((x2*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x3p=max_x-((x2*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y3p=min_y+((x2*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    x4p=max_x-((x1*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y4p=min_y+((x1*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    
    return Polygon(((y1p,x1p),(y2p,x2p),(y3p,x3p),(y4p,x4p),(y1p,x1p)))

#     if df_cache["Tipo"][i]=="casa":
#         casas.append(Polygon([(rectangulo["y1"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x2"][0]),(rectangulo["y1"][0],rectangulo["x2"][0])]))
#     else:
#         terreno.append(Polygon([(rectangulo["y1"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x2"][0]),(rectangulo["y1"][0],rectangulo["x2"][0])]))


## Inicializar modelo entrenado deteccion de objetos basado en yolov7

In [ ]:
os.chdir("C:/Users/ASUS/Inteligencia_Artificial/yolov7")
from detect_Alberto_v2 import modelo
Modelo=modelo(weights="Modelos/best59.pt")

In [ ]:
##test
dim=600
start=(.10,.10)
end=(.20,.20)
proyecciones=[(-11086618.780918632, 2180833.341685466, 0.0),
 (-11086618.780918632, 2180693.108791809, 0.0),
 (-11086478.157352723, 2180693.108791809, 0.0),
 (-11086478.157352723, 2180833.341685466, 0.0)]
rotacion_detect(start,end,2,proyeciones=proyecciones)
testimage=r"train/images/centro_0_0.png"
image=cv2.imread(testimage)
image=cv2.resize(image,(256,256))
verde=verificacion(image)
vector2=[]
vector2=Modelo.detect(opt_source=testimage,opt_conf_thres=0.1)
angulo=0
if (verde <70 and len(vector2)<20):
    angulo = angulo+30
    image_ro=image.copy()
    M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
    image_ro = cv2.warpAffine(image_ro, M, (dim,dim))
    cv2.imwrite("test.png",image_ro)
    vector1=Modelo.detect(opt_source="test.png",opt_conf_thres=0.1)
    if len(vector2)<len(vector1):
        vector2=vector1
        image=image_ro
        print(angulo)
    else:
        vector2=Modelo.detect(opt_source=testimage,opt_conf_thres=0.1)
        print(angulo)
vector2
df_cache=vector2xy(vector2,dim=256)
df_cache["geometry"]=df_cache.apply(lambda x: rotacion_detect(x.start_point_100, x.end_point_100,angulo,proyeciones=proyecciones), axis=1)
gdf=gpd.GeoDataFrame(df_cache[["Tipo","area","imagen"]],geometry=df_cache["geometry"],crs=3857)
gdf.to_file("test.shp")

## Segmentar raster a partir de mascara de capa (normalizada)

In [ ]:
# raster = r"C:\Users\ASUS\Desktop\Centro_Toluca\raster_cortado_centro.tif"
# raster=r"E:\geoshapes\TOLUCA\zonas\Centro.tif"
# raster='mask.tif'
# raster=r"C:\Users\ASUS\Desktop\qgiszonas\test.tif"
raster=r"C:\Users\ASUS\Desktop\naucalpan.tif"
# raster= r"C:/Users/ASUS/Desktop/qgiszonas/centro_nau.tif"
gdal_interpeter = gdal.Open(raster)
width = gdal_interpeter.RasterXSize
height = gdal_interpeter.RasterYSize
coordenadas_gdal = gdal_interpeter.GetGeoTransform()
minx = coordenadas_gdal[0]
miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
maxy = coordenadas_gdal[3] 
minx,maxx,miny,maxy,"W",maxx-minx,"H",maxy-miny

In [ ]:
src_raster_path = raster
src=rasterio.open(src_raster_path)
H,W=src.shape
src.crs

In [ ]:
dim=600
alto=int(np.floor(H/dim))
ancho=int(np.floor(W/dim))
alto,ancho

In [ ]:

nombres=[]
imshow=False
result=pd.DataFrame()
casas=[]
terreno=[]
for j in tqdm.tqdm(range(ancho)):#ancho
    for i in (range(alto)):#alto
        # j=1
        label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
        nameimg=label.lower()+str(i)+"_"+str(j)
        nombres.append(nameimg)
        cuadro=[]
        for k in range(2):
            for l in range(2):
                cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                               maxy-(maxy-miny)/alto*(i+l),
                               0.0))
        cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
        shapes=[{"type":'Polygon','coordinates':[cuadro]}]
        vector=[]
        array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        array=array.copy()
        four_images=[array[2],array[1],array[0],array[3]]
        stacked_images = np.stack(four_images, axis=-1)
        imagen_n=0
        imagen_n=stacked_images.copy()
        verde=verificacion(imagen_n)
        cv2.imwrite("cache.png",imagen_n)
        angulo=correct_orientation("cache.png",dim=dim)
        with torch.no_grad():
            vector=(Modelo.detect(opt_source="cache.png",opt_conf_thres=0.08))
#             if (verde <80 and len(vector)<40):
            image_ro=imagen_n.copy()
            image_ro
            M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
            image_ro = cv2.warpAffine(image_ro, M, (dim,dim))
            cv2.imwrite("cache1.png",image_ro)
            vector1=Modelo.detect(opt_source="cache1.png",opt_conf_thres=0.08)
            if len(vector)<len(vector1):
                vector=vector1
                imagen_n=image_ro
#             print(angulo)
            else:
                angulo=0
# #                     print(angulo)
#                 vector=Modelo.detect(opt_source="cache.png",opt_conf_thres=0.15)
        
        proyecciones=shapes[0].get("coordinates")[0][:-1]
        df_cache=vector2xy(vector,dim=dim,nameimg=nameimg)
        for i in (range(len(df_cache))):
            if df_cache["Tipo"][i]=="casa":
                casas.append(rotacion_detect(df_cache.loc[i,"start_point_100"], df_cache.loc[i,"end_point_100"],-angulo,proyeciones=proyecciones))
            else:
                terreno.append(rotacion_detect(df_cache.loc[i,"start_point_100"], df_cache.loc[i,"end_point_100"],-angulo,proyeciones=proyecciones))
        if imshow:
            imshow_detect(df_cache,imagen_n)

        
# proyecciones

In [ ]:

gdf_casas=gpd.GeoDataFrame(geometry=casas,crs=src.crs)
gdf_casas.set_crs=src.crs
gdf_casas.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7/poligonos_casas_todas_nau.shp")
gdf_casas["area"]=gdf_casas.area
gdf_casas=gdf_casas[gdf_casas["area"]<=gdf_casas["area"].mean()*1.5]
gdf_casas.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7/t1_poligonos_casas_nau1.shp")
gdf_terreno=gpd.GeoDataFrame(geometry=terreno,crs=src.crs)
gdf_terreno.set_crs=src.crs
gdf_terreno.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7/t1_poligonos_terreno_nau1.shp")


In [ ]:
ñ

# crear poligono a partir de coordenada

In [ ]:
gdf=gpd.read_file("C:/Users/ASUS/Desktop/qgiszonas/points.shp")
gdf=gdf.to_crs("3857")
gdf

In [ ]:

nombres=[]
imshow=False
result=pd.DataFrame()
casas=[]
terreno=[]
for c in tqdm.tqdm(gdf["geometry"]):
    x1=int(c.x-30)
    x2=int(c.x+30)
    y1=int(c.y-30)
    y2=int(c.y+30)
    cuadro=[(x1,y1,0),(x1,y2,0),(x2,y2,0),(x2,y1,0),(x1,y1,0)]
    shapes=[{"type":'Polygon','coordinates':[cuadro]}]
    vector=[]
    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    array=array.copy()
    four_images=[array[2],array[1],array[0],array[3]]
    stacked_images = np.stack(four_images, axis=-1)
    imagen_n=0
    imagen_n=stacked_images.copy()
    verde=verificacion(imagen_n)
    cv2.imwrite("cache.png",imagen_n)
    angulo=correct_orientation("cache.png",dim=dim)
    with torch.no_grad():
        vector=(Modelo.detect(opt_source="cache.png",opt_conf_thres=0.08))
#             if (verde <80 and len(vector)<40):
        image_ro=imagen_n.copy()
        image_ro
        M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
        image_ro = cv2.warpAffine(image_ro, M, (dim,dim))
        cv2.imwrite("cache1.png",image_ro)
        vector1=Modelo.detect(opt_source="cache1.png",opt_conf_thres=0.08)
        if len(vector)<len(vector1):
            vector=vector1
            imagen_n=image_ro
#             print(angulo)
        else:
            angulo=0
# #                     print(angulo)
#                 vector=Modelo.detect(opt_source="cache.png",opt_conf_thres=0.15)

    proyecciones=shapes[0].get("coordinates")[0][:-1]
    df_cache=vector2xy(vector,dim=dim,nameimg="none")
    for i in (range(len(df_cache))):
        if df_cache["Tipo"][i]=="casa":
            casas.append(rotacion_detect(df_cache.loc[i,"start_point_100"], df_cache.loc[i,"end_point_100"],-angulo,proyeciones=proyecciones))
        else:
            terreno.append(rotacion_detect(df_cache.loc[i,"start_point_100"], df_cache.loc[i,"end_point_100"],-angulo,proyeciones=proyecciones))
    if imshow:
        imshow_detect(df_cache,imagen_n)

In [ ]:
#os.mkdir("C:/Users/ASUS/Desktop/qgiszonas/salida")
gdf_casas=gpd.GeoDataFrame(geometry=casas,crs=src.crs)
gdf_casas.set_crs=src.crs
gdf_casas.to_file(r"C:/Users/ASUS/Desktop/qgiszonas/salida/casas_t_especificios.shp")
gdf_casas["area"]=gdf_casas.area
gdf_casas=gdf_casas[gdf_casas["area"]<=gdf_casas["area"].mean()*1.5]
gdf_casas.to_file(r"C:/Users/ASUS/Desktop/qgiszonas/salida/casas_t_especificios_1.shp")
gdf_terreno=gpd.GeoDataFrame(geometry=terreno,crs=src.crs)
gdf_terreno.set_crs=src.crs
gdf_terreno.to_file(r"C:/Users/ASUS/Desktop/qgiszonas/salida/terreno_t_especificios_1.shp")


In [ ]:
(-11052512.325050274,
 -11049178.526091522,
 2207425.4937002743,
 2210256.088241955,
 'W',
 3333.7989587523043,
 'H',
 2830.5945416805334)

In [ ]:
##fin
ñ

In [ ]:
gdf_casas=gdf_casas.reset_index(drop=True)
long=gdf_casas.centroid.x
lat=gdf_casas.centroid.y
gdf_casas["latitud"]=lat
gdf_casas["longitud"]=long
gdf_casas.sort_values(by=["latitud"],inplace=True)
dif=[11]
for i in range(len(gdf_casas)-1):
    try:
        dif.append(np.sqrt((long[i+1]-lot[i])**2+(lat[i+1]-lat[i])**2))
    except:
        dif.append(0)
gdf_casas["distancia1"]=dif
gdf_casas.sort_values(by=["longitud"],inplace=True)
dif=[11]
for i in range(len(gdf_casas)-1):
    try:
        dif.append(np.sqrt((long[i+1]-lot[i])**2+(lat[i+1]-lat[i])**2))
    except:
        dif.append(0)
gdf_casas["distancia2"]=dif

In [ ]:
a=gdf_casas#[(gdf_casas["distancia1"]10)]
a
# a=a[(a["distancia2"]>10)]

In [ ]:
a.distancia1.value_counts()

In [ ]:
a=a[['geometry', 'area']]
a.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7/t_poligonos_casas_1.shp")

In [ ]:
a

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
dif.sort()
plt.plot(dif)

In [ ]:
x,y=-11095486.38,2189897.04
x1,y1=-11095486.76,2189886.59
np.sqrt((x1-x)**2+(y1-y)**2)

In [ ]:
gdf_terreno

In [ ]:
print(gdf_casas["geometry"][0])

In [ ]:
gdf_terreno

In [ ]:

33000*3/1360
alto,ancho,alto*ancho,(10*60+46)/60

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
gdf_casas=gpd.GeoDataFrame(geometry=casas,crs=src.crs)
gdf_casas.set_crs=src.crs
gdf_casas["area"]=gdf_casas.area
gdf_casas=gdf_casas[gdf_casas["area"]<=gdf_casas["area"].mean()*2]
gdf_casas
gdf_casas.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7/nuevos_poligonos_casas.shp")

In [ ]:
ñ

In [ ]:
result.loc[0,"start_point_100"],result.loc[0,"end_point_100"]

In [ ]:
proyecciones=shapes[0].get("coordinates")[0][:-1]
np.min((proyecciones,proyecciones),axis=1)[0],np.max((proyecciones,proyecciones),axis=1)[0]

In [ ]:
proyecciones=shapes[0].get("coordinates")[0][:-1]
min_y,min_x,_=np.min((proyecciones,proyecciones),axis=1)[0]
max_y,max_x,_=np.max((proyecciones,proyecciones),axis=1)[0]
min_y,min_x,max_y,max_x,proyecciones

tipos=["casa","terreno"]
geometry=[]
for tipo in tipos:
    df_aux=df_cache[df_cache["Tipo"]==tipo].reset_index(drop=True)
    for i in tqdm.tqdm(range(len(df_aux))):
        coordenadas=[(df_aux["start_point_100"][i][1],df_aux["start_point_100"][i][0],df_aux["end_point_100"][i][1],df_aux["end_point_100"][i][0])]
        rectangulo=pd.DataFrame(coordenadas,columns=["x1","y1","x2","y2"])
        rectangulo["x1"]=max_x-rectangulo["x1"]*(max_x-min_x)
        rectangulo["y1"]=min_y+rectangulo["y1"]*(max_y-min_y)
        rectangulo["x2"]=max_x-rectangulo["x2"]*(max_x-min_x)
        rectangulo["y2"]=min_y+rectangulo["y2"]*(max_y-min_y)
        geometry.append(Polygon([(rectangulo["y1"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x1"][0]),(rectangulo["y2"][0],rectangulo["x2"][0]),(rectangulo["y1"][0],rectangulo["x2"][0])]))
# geometry
    gdf1=gpd.GeoDataFrame(geometry=geometry,crs=src.crs)
    gdf1.set_crs=src.crs
    gdf1.to_file(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7/nuevos_poligonos_"+tipo+".shp")


In [ ]:
geometry[1]

In [ ]:
proyecciones=shapes[0].get("coordinates")[0][:-1]
miny,minx=proyecciones[0][:-1]
maxy,maxx=proyecciones[-1][:-1]

In [ ]:
vector=[]
array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
# array=np.moveaxis(out_image,0,-1)
four_images=[array[2],array[1],array[0]]
stacked_images = np.stack(four_images, axis=-1)
cv2.imwrite("cache.png",stacked_images)
vector=(Modelo.detect(opt_source="cache.png",opt_conf_thres=0.1))
vector


In [ ]:
s=[]
for v in vector:
    str_v=(str(v).replace("tensor(","").replace("[","").replace("]","").replace(".)","").replace(")","").replace("(","").replace("']","").replace("'","").strip().split(","))
    h,w=1024,1024
    x1 = int( float(str_v[1]) * w )
    y1 = int( float(str_v[2]) * h )
    xw = int( float(str_v[3]) * w /2)
    yw = int( float(str_v[4]) * h /2)
    start_point = ((x1 - xw), (y1 - yw) )
    end_point   = ((x1 + xw), (y1 + yw) )
    if str(str_v[0])=="0":
        tipo="casa"
    else:
        tipo="terreno"
    s.append([tipo,start_point,end_point])
resultado_nogeo=pd.DataFrame(s,columns=["Tipo","start_point","end_point"])
resultado_nogeo.drop_duplicates()

In [ ]:
cv2.imshow("s",stacked_images)
cv2.waitKey()

In [ ]:
for i in range(len(resultado_nogeo)):
    try:
        cv2.rectangle(stacked_images,resultado_nogeo["start_point"][i],resultado_nogeo["end_point"][i],(0,255,0),2)
    except:pass
cv2.imshow("s",stacked_images)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
resultado_nogeo["start_point"][i]

In [ ]:
import cv2
import numpy as np

!wget  https://i.stack.imgur.com/sDQLM.png
#read image 
image = cv2.imread(testimage)

#convert to gray
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#performing binary thresholding
kernel_size = 10
ret,thresh = cv2.threshold(gray,100,120,cv2.THRESH_BINARY)  

#finding contours 
cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

#drawing Contours
radius =1
color = (30,255,50)
cv2.drawContours(image, cnts, -1,color , radius)
# cv2.imshow(image) commented as colab don't support cv2.imshow()
cv2.imshow("1",image)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
img = cv2.imread(testimage,0)
edges = cv2.Canny(img,100,400)
edges=cv2.dilate(edges,(100,100))
cv2.imshow("h",edges)
cv2.waitKey()
cv2.destroyAllWindows()

# plt.subplot(121),plt.imshow(img,cmap = 'gray')
# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(122),plt.imshow(edges,cmap = 'gray')
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
# plt.show()

In [ ]:
import cv2 as cv
from math import atan2, cos, sin, sqrt, pi
import numpy as np
 
def drawAxis(img, p_, q_, color, scale):
  p = list(p_)
  q = list(q_)
 
  ## [visualization1]
  angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
  hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))
 
  # Here we lengthen the arrow by a factor of scale
  q[0] = p[0] - scale * hypotenuse * cos(angle)
  q[1] = p[1] - scale * hypotenuse * sin(angle)
  cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)
 
  # create the arrow hooks
  p[0] = q[0] + 9 * cos(angle + pi / 4)
  p[1] = q[1] + 9 * sin(angle + pi / 4)
  cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)
 
  p[0] = q[0] + 9 * cos(angle - pi / 4)
  p[1] = q[1] + 9 * sin(angle - pi / 4)
  cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)
  ## [visualization1]
 
def getOrientation(pts, img):
  ## [pca]
  # Construct a buffer used by the pca analysis
  sz = len(pts)
  data_pts = np.empty((sz, 2), dtype=np.float64)
  for i in range(data_pts.shape[0]):
    data_pts[i,0] = pts[i,0,0]
    data_pts[i,1] = pts[i,0,1]
 
  # Perform PCA analysis
  mean = np.empty((0))
  mean, eigenvectors, eigenvalues = cv.PCACompute2(data_pts, mean)
 
  # Store the center of the object
  cntr = (int(mean[0,0]), int(mean[0,1]))
  ## [pca]
 
  ## [visualization]
  # Draw the principal components
  cv.circle(img, cntr, 3, (255, 0, 255), 2)
  p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
  p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
  drawAxis(img, cntr, p1, (255, 255, 0), 1)
  drawAxis(img, cntr, p2, (0, 0, 255), 5)
 
  angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
  ## [visualization]
 
  # Label with the rotation angle
  label = "  Rotation Angle: " + str(-int(np.rad2deg(angle)) - 90) + " degrees"
  textbox = cv.rectangle(img, (cntr[0], cntr[1]-25), (cntr[0] + 250, cntr[1] + 10), (255,255,255), -1)
  cv.putText(img, label, (cntr[0], cntr[1]), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv.LINE_AA)
 
  return angle
 
# Load the image
img = cv.imread(raster)
# img=imagen_n
# Was the image there?
if img is None:
  print("Error: File not found")
  exit(0)
 
cv.imshow('Input Image', img)
 
# Convert image to grayscale
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

gray=cv2.blur(gray,(10,10))
# Convert image to binary
_, bw = cv.threshold(gray, 200, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
bw=array[3]
# Find all the contours in the thresholded image
contours, _ = cv.findContours(bw, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
 
for i, c in enumerate(contours):
 
  # Calculate the area of each contour
  area = cv.contourArea(c)
 
  # Ignore contours that are too small or too large
  if area < 200000:
    continue
 
  # Draw each contour only for visualisation purposes
  cv.drawContours(img, contours, i, (0, 0, 255), 2)
 
  # Find the orientation of each shape
  getOrientation(c, img)
 
cv.imshow('Output Image', img)
cv.waitKey(0)
cv.destroyAllWindows()
  
# Save the output image to the current directory
# cv.imwrite("output_img.jpg", img)

In [ ]:
bw

In [ ]:
a=gpd.read_file("E:\geoshapes\TOLUCA\TOLUCA\Manzana_Toluca.shp")

In [ ]:
a=a.to_crs(3857)

In [ ]:
shapes=mapping(a)


In [ ]:
raster=r"E:\geoshapes\TOLUCA\zonas\Centro.tif"
raster=r"C:\Users\ASUS\Desktop\qgiszonas\test.tif"
gdal_interpeter = gdal.Open(raster)
width = gdal_interpeter.RasterXSize
height = gdal_interpeter.RasterYSize
coordenadas_gdal = gdal_interpeter.GetGeoTransform()
minx = coordenadas_gdal[0]
miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
maxy = coordenadas_gdal[3] 
minx,maxx,miny,maxy,"W",maxx-minx,"H",maxy-miny

In [ ]:
b=mapping(a["geometry"])

In [ ]:
c=b["features"][0].get('geometry').get("coordinates")[0]


In [ ]:
distancia=[]
pendiente=[]
c=list(c)
for i in range(len(c)-1):
    x,y,z=c[i]
    x1,y1,z1=c[i+1]
    distancia.append(math.sqrt((x1-x)**2+(y1-y)**2))
    pendiente.append((y1-y)/(x1-x))
    

In [ ]:
distancia 

In [ ]:
pendiente

In [ ]:
src_raster_path = raster
src=rasterio.open(src_raster_path)
H,W=src.shape
src.crs

In [ ]:
import fiona
shp_file_path = r"C:\Users\ASUS\Desktop\Manzana_Toluca_modified.shp"
shapefile=fiona.open(shp_file_path, "r")
shape=[feature["geometry"] for feature in shapefile]
# shapes=shapefile[3]["geometry"]
# shapes=[shapes,]

shape

In [ ]:
array, out_transform = rasterio.mask.mask(src, [shapes], crop=True)
array

In [ ]:
import fiona
import rasterio
from rasterio.mask import mask

def create_mask_from_shapefile(shapefile_filepath, corresponding_orthomosaic_filepath):

    # open shapefile
    with fiona.open(shapefile_filepath, 'r') as shapefile:
        shapes = [feature['geometry'] for feature in shapefile]

    # open rasterfile
    with rasterio.open(corresponding_orthomosaic_filepath, 'r') as src:
        out_image, out_transform = mask(src, shapes, crop=True) # setting all pixels outside of the feature zone to zero
        out_meta = src.meta

    out_meta.update({"driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform})

    output_file = 'mask.tif'

    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(out_image)

input_raster = r"C:\Users\ASUS\Desktop\qgiszonas\test.tif"
input_shapefile = r"C:\Users\ASUS\Desktop\Manzana_Toluca_modified.shp"

create_mask_from_shapefile(input_shapefile, input_raster)

In [ ]:
correct_orientation(img_rgb)

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
img_rgb = cv.imread(r"train/images/centro_0_28.png")
img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
template = cv.imread("pattern.png",0)
# template=np.ones((1000,1000),"uint8")*255
w, h = template.shape[::-1]
dim=img_rgb.shape[0]
image_ro=img_gray.copy()
angulo=0
an=[]
le=0
angulo_f=0
for i in range(90):
    angulo+=1
    M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
    image_ro = cv2.warpAffine(img_gray, M, (dim,dim))
    res = cv.matchTemplate(image_ro,template,cv.TM_CCOEFF_NORMED)
    threshold =.5
    loc = np.where( res >= threshold)
    com=len(loc[0])
    if com>0:
        an.append(angulo)
        if le<com:
            le=com
            angulo_f=angulo
angulo=angulo_f
M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
image_ro = cv2.warpAffine(img_rgb, M, (dim,dim))
img_gray = cv.cvtColor(image_ro, cv.COLOR_BGR2GRAY)
res = cv.matchTemplate(img_gray,template,cv.TM_CCOEFF_NORMED)
threshold =.7
loc = np.where( res >= threshold)
# for pt in zip(*loc[::-1]):
#     cv.rectangle(image_ro, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
print(angulo_f)
# cv.imshow('res.png',image_ro)
# cv.waitKey()
# cv2.destroyAllWindows()
cv.imwrite('res.png',image_ro)

In [ ]:
imshow_detect(vector2xy(Modelo.detect(opt_conf_thres=0.1,opt_source="res.png"),dim=1024,angle=angulo_f),image_ro)

In [ ]:
np.zeros((10,512),"uint8")
# template

In [ ]:
template.shape

